In [12]:
import escher
import cobra
import cobrakbase
import logging

%run ../../../scripts/bios_utils.py
%run ../../../scripts/escher_curation.py
%run ../../../scripts/sbml_func.py

logger = logging.getLogger(__name__)
kbase = cobrakbase.KBaseAPI('L3NXWHHX25VLEL6OAWB3Z5URDSDKZIMP', dev=True)

In [2]:
modelseed_local = cobrakbase.modelseed.from_local('/Users/fliu/workspace/jupyter/ModelSEEDDatabase')

load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/reactions.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/compounds.tsv
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: /Users/fliu/workspace/jupyter/ModelSEEDDatabase/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [2]:
ws = 'filipeliu:narrative_1556512034170'
kmodel = kbase.get_object('GCF_000005845.2.core.mdl', ws)
kmedia = kbase.get_object('Carbon-D-Glucose', ws)
kfba = kbase.get_object('GCF_000005845.2.core.fba', ws)
ref = kbase.get_object_info_from_ref(kmodel['genome_ref'])
kgenome = kbase.get_object(ref.id, ref.workspace_id)

fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)
media = cobrakbase.core.KBaseBiochemMedia(kmedia)
genome = cobrakbase.core.KBaseGenome(kgenome)
fba = cobrakbase.core.KBaseFBA(kfba)

builder = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(
    fbamodel).with_media(media).with_genome(genome)
cobra_model = builder.build()
cobra_model.objective = "bio1_biomass"
model_json = cobra.io.to_json(cobra_model)

In [16]:
genome.data.keys()

dict_keys(['assembly_ref', 'cdss', 'dna_size', 'domain', 'external_source_origination_date', 'feature_counts', 'features', 'gc_content', 'genbank_handle_ref', 'genetic_code', 'genome_tiers', 'id', 'md5', 'molecule_type', 'mrnas', 'non_coding_features', 'notes', 'num_contigs', 'ontologies_present', 'ontology_events', 'original_source_file_name', 'publications', 'reference_annotation', 'scientific_name', 'source', 'source_id', 'taxon_ref', 'taxonomy', 'type', 'warnings'])

In [20]:
%run ../../../cobrakbase/cobrakbase/core/converters.py
builder = KBaseFBAModelToCobraBuilder(fbamodel, media_const)
cobra_model = builder.build()

In [4]:
ws = 'filipeliu:narrative_1504708269077'
kmodel = kbase.get_object('kb.iJO1366_bigg2', ws)
media = cobrakbase.core.KBaseBiochemMedia(kbase.get_object('iJO1366_bigg2.media', ws))
fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)
builder = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(fbamodel).with_media(media)
cobra_model = builder.build()
add_cytosol_sinks(cobra_model, DM_iJO1366) #sbml_func.py
model_json = cobra.io.to_json(cobra_model)
solution = cobra_model.optimize()
print('default growth', solution) #default growth <Solution 1.009 at 0x1331ea1eb8>

default growth <Solution 1.009 at 0x1329bce630>


In [3]:
ws = 'filipeliu:narrative_1504708269077'
kmedia = kbase.get_object('iJO1366_bigg2.media', ws)
media = cobrakbase.core.KBaseBiochemMedia(kmedia)

In [417]:
#modelseed_local.reaction_aliases['rxn00001']
bigg_to_seed = {}
for seed_id in modelseed_local.reaction_aliases:
    if 'BiGG' in modelseed_local.reaction_aliases[seed_id]:
        for db_id in modelseed_local.reaction_aliases[seed_id]['BiGG']:
            if not db_id in bigg_to_seed:
                bigg_to_seed[db_id] = set()
            bigg_to_seed[db_id].add(seed_id)

In [422]:
sorted(bigg_to_seed[rxn.id[2:]])[0]

'rxn01791'

In [470]:
for rxn in fbamodel.reactions:
    rxn.data['dblinks']['sbml_id'] = rxn.id
    if rxn.id[2:] in bigg_to_seed:
        rxn.data['dblinks']['seed.reaction'] = sorted(bigg_to_seed[rxn.id[2:]])[0]

In [471]:
used = set()
for rxn in fbamodel.reactions:
    if 'seed.reaction' in rxn.data['dblinks']:
        cmp = get_reaction_compartment(get_compartments(rxn))
        seed_id = rxn.data['dblinks']['seed.reaction']
        id = seed_id + '_' + cmp
        copy_counter = 1
        if not id in used:
            rxn.data['id'] = id
            used.add(id)
        else:
            while id in used:
                id = seed_id + '_copy' + str(copy_counter) + '_' + cmp
                copy_counter += 1
            rxn.data['id'] = id
            used.add(id)
        rxn.data['reaction_ref'] = '50/1/1/reactions/id/' + seed_id + '_' + cmp
rxn.data

{'aliases': [],
 'dblinks': {'sbml_id': 'R_DKGLCNR1', 'seed.reaction': 'rxn08345'},
 'direction': '=',
 'id': 'rxn08345_c0',
 'imported_gpr': '(b0207 or b3012)',
 'maxforflux': 1000.0,
 'maxrevflux': 0.0,
 'modelReactionProteins': [{'complex_ref': '',
   'modelReactionProteinSubunits': [{'feature_refs': ['8416/7/1/features/id/b3012',
      '8416/7/1/features/id/b0207'],
     'note': 'Imported GPR',
     'optionalSubunit': 0,
     'role': '',
     'triggering': 0}],
   'note': '',
   'source': 'SBML'}],
 'modelReactionReagents': [{'coefficient': -1.0,
   'modelcompound_ref': '~/modelcompounds/id/cpd01793_c0'},
  {'coefficient': -1.0,
   'modelcompound_ref': '~/modelcompounds/id/cpd00067_c0'},
  {'coefficient': -1.0,
   'modelcompound_ref': '~/modelcompounds/id/cpd00005_c0'},
  {'coefficient': 1.0, 'modelcompound_ref': '~/modelcompounds/id/cpd15351_c0'},
  {'coefficient': 1.0,
   'modelcompound_ref': '~/modelcompounds/id/cpd00006_c0'}],
 'modelcompartment_ref': '~/modelcompartments/id/c0

In [423]:
for rxn in fbamodel.reactions:
    rxn.data['dblinks']['sbml_id'] = rxn.id
    #modelseed_local.
    print(rxn.id, bigg_to_seed[rxn.id[2:]])
    break
rxn.data

R_PANTS {'rxn35352', 'rxn01791'}


{'aliases': [],
 'dblinks': {'sbml_id': 'R_PANTS'},
 'direction': '=',
 'id': 'R_PANTS',
 'imported_gpr': 'b0133',
 'maxforflux': 1000.0,
 'maxrevflux': 0.0,
 'modelReactionProteins': [{'complex_ref': '',
   'modelReactionProteinSubunits': [{'feature_refs': ['8416/7/1/features/id/b0133'],
     'note': 'Imported GPR',
     'optionalSubunit': 0,
     'role': '',
     'triggering': 0}],
   'note': '',
   'source': 'SBML'}],
 'modelReactionReagents': [{'coefficient': -1.0,
   'modelcompound_ref': '~/modelcompounds/id/cpd00085_c0'},
  {'coefficient': -1.0,
   'modelcompound_ref': '~/modelcompounds/id/cpd00002_c0'},
  {'coefficient': -1.0,
   'modelcompound_ref': '~/modelcompounds/id/cpd00408_c0'},
  {'coefficient': 1.0, 'modelcompound_ref': '~/modelcompounds/id/cpd00018_c0'},
  {'coefficient': 1.0, 'modelcompound_ref': '~/modelcompounds/id/cpd00067_c0'},
  {'coefficient': 1.0, 'modelcompound_ref': '~/modelcompounds/id/cpd00644_c0'},
  {'coefficient': 1.0,
   'modelcompound_ref': '~/modelcom

In [347]:
ws = 'filipeliu:narrative_1504192868437'


In [226]:
kbase.save_object(id='Lipst1_1_NO_GP_DraftFilter.gf.Carbon-D-Glucose.fix', ws=ws, data=kmodel, otype='KBaseFBA.FBAModel')

[[283,
  'Lipst1_1_NO_GP_DraftFilter.gf.Carbon-D-Glucose.fix',
  'KBaseFBA.FBAModel-11.0',
  '2019-06-17T07:14:22+0000',
  1,
  'filipeliu',
  40036,
  'filipeliu:narrative_1548978605026',
  '8213bef0144e0d15dab2af333a4ff25e',
  1743918,
  {'Number gapgens': '0',
   'Type': 'MultipleGenomes',
   'Number gapfills': '1',
   'Source ID': 'fungi_merge_model_empty',
   'Number biomasses': '1',
   'Number compartments': '9',
   'Genome': '39957/29/5',
   'Source': 'sbml',
   'Number compounds': '2486',
   'Number reactions': '632',
   'Name': 'Lipomyces starkeyi'}]]

In [370]:
cobra_model2 = cobra.io.read_sbml_model('../../../data/sbml/iJO1366.xml')

In [472]:
media_const = cobrakbase.convert_media(kmedia)
cobra_model = cobrakbase.convert_kmodel(kmodel, media_const)
model_json = cobra.io.to_json(cobra_model)

In [379]:
cobra_model2.demands[0]

Reaction identifier,DM_4crsol_c
Name,Sink needed to allow p-Cresol to leave system
Memory address,0x0135b5b1d30
Stoichiometry,4crsol_c --> P-Cresol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [240]:
ws = 'filipeliu:narrative_1548978605026'
kmodel = kbase.get_object('Lipst1_1_NO_GP_DraftFilter.gf.Carbon-D-Glucose.fix', ws)
kmedia = kbase.get_object('Carbon-D-Glucose', ws)
#kgenome = 
kfba = kbase.get_object('Lipst1_1_NO_GP_DraftFilter.gf.fba', ws)
fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)

In [493]:
#ecoli
kmodel = kbase.get_object('GCF_000005845.2.mdl', ws)
kmedia = kbase.get_object('Carbon-D-Glucose', ws)
#kgenome = 
kfba = kbase.get_object('GCF_000005845.2.fba', ws)

In [518]:
print(cobra_model.reactions[0])

rxn02201_c0: cpd00443_c0 + cpd02920_c0 --> cpd00012_c0 + cpd00067_c0 + cpd00683_c0


In [521]:
seed_coefficients_to_string(fbamodel.reactions[0].data['modelReactionReagents'])

{'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00443_c0'}
{'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd02920_c0'}
{'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00012_c0'}
{'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00067_c0'}
{'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00683_c0'}


'cpd00443_c0 + cpd02920_c0 <=> cpd00012_c0 + cpd00067_c0 + cpd00683_c0'

In [513]:
def seed_coefficients_to_string(coeff_list, op = '<=>'):
    lhs = []
    rhs = []
    pair_to_str = lambda v, id : id if math.fabs(v) == 1 else str(math.fabs(v)) + ' ' + id
    for o in coeff_list:
        cpd_id = o['modelcompound_ref'].split('/')[-1]
        value = o['coefficient']
        if value > 0:
            rhs.append(pair_to_str(value, cpd_id))
        elif value < 0:
            lhs.append(pair_to_str(value, cpd_id))
        else:
            pass
    return "{} {} {}".format(' + '.join(lhs), op, ' + '.join(rhs))

for o in kmodel['biomasses']:
    print(seed_coefficients_to_string(o['biomasscompounds']))

{'coefficient': 41.257, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd00008_c0'}
{'coefficient': 3.547, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd00004_c0'}
{'coefficient': -0.8977, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd00101_c0'}
{'coefficient': -0.8977, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd00236_c0'}
{'coefficient': -1.8225, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd00005_c0'}
{'coefficient': 1, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd11416_c0'}
{'coefficient': -0.0709, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd00072_c0'}
{'coefficient': 41.257, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id/cpd00009_c0'}
{'coefficient': -0.129, 'edits': {}, 'gapfill_data': {}, 'modelcompound_ref': '~/modelcompounds/id

In [174]:
%run ../../../scripts/stoich_integration.py

In [432]:
def get_reaction_compartment(cmps):
    if not cmps == None:
        if len(cmps) == 1:
            return cmps.pop()
        elif 'e0' in cmps and len(cmps) == 2:
            return 'e0'
        elif 'r0' in cmps and len(cmps) == 2:
            return 'r0'
        elif 'c0' in cmps and len(cmps) == 2:
            cmps.remove('c0')
            return cmps.pop()
        elif 'p0' in cmps and len(cmps) == 3:
            return 'p0'
        elif len(cmps) == 0:
            print(reaction)
        print(cmps)
    else:
        return None

def get_compartments(r):
    compartments = set()
    for o in r.data['modelReactionReagents']:
        cpd_id = o['modelcompound_ref'].split('/')[-1]
        cpd = fbamodel.get_metabolite(cpd_id)
        cpd_cmp = cpd.data['modelcompartment_ref'].split('/')[-1]
        compartments.add(cpd_cmp)
    return compartments

def get_cstoichiometry(r):
    cstoichiometry = {}
    for o in r.data['modelReactionReagents']:
        cpd_id = o['modelcompound_ref'].split('/')[-1]
        cpd = fbamodel.get_metabolite(cpd_id)
        cpd_cmp = cpd.data['modelcompartment_ref'].split('/')[-1]
        coeff = o['coefficient']
        cstoichiometry[(cpd.id, cpd_cmp)] = coeff
    return cstoichiometry
        
#get_cs(r)

In [228]:
for i in range(len(kmodel['modelreactions'])):
    r = kmodel['modelreactions'][i]
    r['id'] = r['id'] + '_' + str(i)

In [229]:
def cluster_reactions2(rxn_to_hash, all_hashes):

    g = nx.Graph()
    for h in all_hashes:
        for h_val in all_hashes[h]:
            #print(h, h_val, all_hashes[h][h_val])
            for i in all_hashes[h][h_val]:
                g.add_edge(str(h_val) + '@HASH', i)
                    #print(prev, i)
            #break

    ccs = nx.algorithms.connected_components(g)
    ccs = [cc for cc in ccs]
    ccs = filter_hash(ccs)
    print('clusters:', len(ccs))
    
    return ccs

ccs = cluster_reactions2(rxn_to_hash, all_hashes)



clusters: 7


In [236]:
for cc in ccs:
    print()
    for id in cc:
        r = fbamodel.get_reaction(id)
        r.data['direction'] = '='
        r.data['maxforflux'] = 1000
        r.data['maxrevflux'] = 1000
        print(r.id, r.stoichiometry, r.get_bounds())


rxn00832_c_369 {'cpd00001_c0': -1, 'cpd00114_c0': -1, 'cpd02884_c0': 1} (1000, 1000, '=')
rxn00832_c_0 {'cpd00001_c0': -1, 'cpd00114_c0': -1, 'cpd02884_c0': 1} (1000, 1000, '=')

rxn00611_c_390 {'cpd00003_c0': -1, 'cpd00080_c0': -1, 'cpd00004_c0': 1, 'cpd00067_c0': 1, 'cpd00095_c0': 1} (1000, 1000, '=')
rxn00611_c_3 {'cpd00003_c0': -1, 'cpd00080_c0': -1, 'cpd00004_c0': 1, 'cpd00067_c0': 1, 'cpd00095_c0': 1} (1000, 1000, '=')

rxn03975_c_488 {'cpd00006_c0': -1, 'cpd01270_c0': -1, 'cpd00005_c0': 1, 'cpd00050_c0': 1, 'cpd00067_c0': 2} (1000, 1000, '=')
rxn03975_c_121 {'cpd00006_c0': -1, 'cpd01270_c0': -1, 'cpd00005_c0': 1, 'cpd00050_c0': 1, 'cpd00067_c0': 2} (1000, 1000, '=')

rxn00086_c_276 {'cpd00006_c0': -1, 'cpd00042_c0': -2, 'cpd00005_c0': 1, 'cpd00067_c0': 1, 'cpd00111_c0': 1} (1000, 1000, '=')
rxn00086_c_191 {'cpd00006_c0': -1, 'cpd00042_c0': -2, 'cpd00005_c0': 1, 'cpd00067_c0': 1, 'cpd00111_c0': 1} (1000, 1000, '=')

rxn00086_m_192 {'cpd00006_m0': -1, 'cpd00042_m0': -2, 'cpd00005

{'aliases': [],
 'dblinks': {},
 'direction': '<',
 'edits': {},
 'gapfill_data': {'gf.0': {'0': ['<', 1, []]}},
 'id': 'rxn00225_c0_537',
 'maxforflux': 1000000,
 'maxrevflux': 1000000,
 'modelReactionProteins': [],
 'modelReactionReagents': [{'coefficient': -1,
   'modelcompound_ref': '~/modelcompounds/id/cpd00002_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00196_c0'},
  {'coefficient': -1, 'modelcompound_ref': '~/modelcompounds/id/cpd00029_c0'},
  {'coefficient': 1, 'modelcompound_ref': '~/modelcompounds/id/cpd00008_c0'}],
 'modelcompartment_ref': '~/modelcompartments/id/c0',
 'name': 'ATP:acetate phosphotransferase_c0',
 'numerical_attributes': {},
 'probability': 0,
 'protons': 0,
 'reaction_ref': '~/template/reactions/id/rxn00225_c',
 'string_attributes': {}}

In [224]:
for i in range(len(kmodel['modelreactions'])):
    r = kmodel['modelreactions'][i]
    r['id'] = rename[r['id']]

In [221]:
delete = set()
for cc in ccs:
    cc.pop()
    delete |= cc
fbamodel.delete_reactions(delete, store=True)
rename = {}
for rxn in fbamodel.reactions:
    seed_id = rxn.data['reaction_ref'].split('/')[-1].split('_')[0]
    cmps = get_compartments(rxn)
    cmp = None
    cmp = get_reaction_compartment(cmps)
    rename[rxn.id] = "{}_{}".format(seed_id, cmp)
    

(632, 632)

In [ ]:
for r in fbamodel.reactions:

In [182]:
h_func = hasher
rxn_to_hash = {}
all_hashes = {}
for r in fbamodel.reactions:
    cstoichiometry = get_cs(r)
    hashes = h_func(cstoichiometry)
    rxn_to_hash[r.id] = hashes
    for h in hashes:
        hash_val = hashes[h]
        if not h in all_hashes:
            all_hashes[h] = {}
        if not hash_val in all_hashes[h]:
            all_hashes[h][hash_val] = set()
        all_hashes[h][hash_val].add(r.id)

In [142]:
fbamodel.get_reaction('rxn00086_c')

In [151]:
fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)
ids = set()
print(len(fbamodel.reactions))
for r in fbamodel.reactions:
    ids.add(r.id)
    if 'rxn00086_c' in r.id:
        print(r.id)

639


In [241]:
media_const = cobrakbase.convert_media(kmedia)
cobra_model = cobrakbase.convert_kmodel(kmodel, media_const)
model_json = cobra.io.to_json(cobra_model)
fbamodel = cobrakbase.core.model.KBaseFBAModel(kmodel)
cobra_model

Name,kbase
Memory address,0x0133fcfa978
Number of metabolites,935
Number of reactions,833
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,"c, m, x, e, v, r"


In [104]:
fba = cobrakbase.core.model.KBaseFBASolution(kfba)
fba

In [243]:
flux_data = {'rxn01643_c0' : -100}
for rxn_var in fba.data['FBAReactionVariables']:
    flux = rxn_var['value']
    rxn_id = rxn_var['modelreaction_ref'].split('/')[-1]
    flux_data[rxn_id] = flux
for rxn_var in fba.data['FBABiomassVariables']:
    flux = rxn_var['value']
    rxn_id = rxn_var['biomass_ref'].split('/')[-1]
    flux_data[rxn_id + '_biomass'] = flux
for rxn_var in fba.data['FBACompoundVariables']:
    flux = rxn_var['value']
    cpd_id = rxn_var['modelcompound_ref'].split('/')[-1]
    flux_data['EX_' + cpd_id] = -1 * flux

In [476]:
def get_flux(cobra_model):
    flux_data = {}
    solution = cobra_model.optimize()
    for rxn in cobra_model.reactions:
        flux_data[rxn.id] = solution.fluxes[rxn.id]
    return flux_data
flux_data = get_flux(cobra_model)
#flux_data

In [245]:
map_list

[{'organism': 'BIOS', 'map_name': 'bios7.proteins'},
 {'organism': 'BIOS', 'map_name': 'bios7.Quinones'},
 {'organism': 'BIOS', 'map_name': 'bios7.fa'},
 {'organism': 'BIOS', 'map_name': 'bios7.AA'},
 {'organism': 'BIOS', 'map_name': 'bios7.LipidIV'},
 {'organism': 'BIOS', 'map_name': 'bios7.thf'},
 {'organism': 'BIOS', 'map_name': 'bios7.Central'},
 {'organism': 'BIOS', 'map_name': 'bios7.mycolate'},
 {'organism': 'BIOS', 'map_name': 'bios7.Riboflavin'},
 {'organism': 'BIOS', 'map_name': 'bios7.sugars'},
 {'organism': 'BIOS', 'map_name': 'bios7.f430'},
 {'organism': 'BIOS', 'map_name': 'bios7.Thiamine'},
 {'organism': 'BIOS', 'map_name': 'bios7.Nucleotides'},
 {'organism': 'BIOS', 'map_name': 'bios7.peptidoglycan'}]

In [29]:
ESCHER_HOME = '/Users/fliu/workspace/jupyter/data/escher'
model_base = 'BIOS'
model_file = 'bios7.json'
escher_model_data = read_json(ESCHER_HOME + '/models/' + model_base + '/' + model_file)
escher_model = EscherModel(escher_model_data)


0 142


In [81]:
ESCHER_HOME = '/Users/fliu/workspace/jupyter/data/escher'
model_base = 'ModelSEED'
model_file = 'ModelSEED.json'
escher_model_data = read_json(ESCHER_HOME + '/models/' + model_base + '/' + model_file)
escher_model = EscherModel(escher_model_data)

In [89]:
for model_reaction in escher_model_data['reactions']:
    if not 'dblinks' in model_reaction:
        model_reaction['dblinks'] = {}
    if not 'seed.reaction' in model_reaction['dblinks']:
        model_reaction['dblinks']['seed.reaction'] = [model_reaction['id']]

In [90]:
write_json(escher_model_data, ESCHER_HOME + '/models/' + model_base + '/' + model_file)

In [37]:
cobra_model_json = cobra.io.to_json(cobra_model)

In [18]:
rxn = kbase_escher.fbamodel.reactions[0]
print(rxn.)

rxn01975_c0: cpd00006_c0 + cpd00863_c0 --> cpd00005_c0 + cpd00067_c0 + cpd00911_c0


In [13]:
kbase_escher = KBaseEscherViewer(fbamodel, 'ModelSEED', 'ModelSEED.json', fba)
kbase_escher.load_model()
kbase_escher.generate_maps('c')
#kbase_escher.save_maps('')

In [28]:
kbase_escher.compartment_layer['c']['cpd_remap']

{}

In [5]:
cpd_map_to_model, rxn_map_to_model = kbase_escher.fetch_metabolites_and_reactions(kbase_escher.fbamodel)
cpd_remap, rxn_remap = kbase_escher.escher_model.map_escher_model_data(cpd_map_to_model, rxn_map_to_model)

In [14]:
builder = kbase_escher.maps['ModelSEED.TCA Cycle'][1]

In [15]:

builder.display_in_notebook()

{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Alginate Biosynthesis'}
10 20 0
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Isoprenoid Biosynthesis'}
24 40 1
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Allantoin Utilization'}
11 18 0
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.TCA Cycle'}
17 33 14
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Poly-Beta-Hydroxybutyrate Metabolism (2)'}
14 21 2
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.NAD(P) Biosynthesis'}
25 43 1
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Coenzyme A Biosynthesis'}
10 25 0
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Arginine and Ornithine Degradation'}
20 31 0
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Threonine Degradation'}
6 17 0
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Methionine Salvage'}
16 29 0
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Catechol Degradation to Beta-Ketoadipate'}
6 13 0
{'organism': 'ModelSEED', 'map_name': 'ModelSEED.Acetyl-CoA Fermentation

In [40]:


def build_escher_map(escher_map, model_json):
    map_json_str = json.dumps(escher_map.escher_map)
    builder = escher.Builder(map_json=map_json_str, model_json=model_json, reaction_data=flux_data)
    builder.set_highlight_missing(True)
    builder.set_enable_tooltips(True)
    builder.set_show_gene_reaction_rules(True)
    builder.set_and_method_in_gene_reaction_rule(True)
    return builder

In [42]:
cpd_map_to_model, rxn_map_to_model = fetch_metabolites_and_reactions(fbamodel, compartment_match2='c')
print(len(rxn_map_to_model), len(cpd_map_to_model))

149 142


In [44]:
cpd_remap, rxn_remap = escher_model.map_escher_model_data(cpd_map_to_model, rxn_map_to_model)
print(len(cpd_remap), len(rxn_remap))

142 125


In [54]:
rxn_in_map = escher_map.reactions

In [192]:
escher_map.swap_ids(cpd_remap, rxn_remap)

In [59]:
map_list = list(filter(lambda d : d['organism'] == model_base, escher.list_cached_maps()))
for o in map_list:
    print(o)
    map_id = o['map_name']
    escher_map_data = read_json(ESCHER_HOME + '/maps/' + model_base + '/' + map_id + '.json')
    escher_map = EscherMap(escher_map_data)
    rxn_in_map = set(map(lambda o : o['bigg_id'], escher_map.reactions))
    print(len(rxn_in_map), len(rxn_in_map & set(rxn_remap)))
    escher_map.swap_ids(cpd_remap, rxn_remap)
    #builder = build_escher_map(escher_map, model_json)
    #builder.save_html('../bin/escher/maps/' + map_id, overwrite=True)

{'organism': 'BIOS', 'map_name': 'bios7.proteins'}
192 1
{'organism': 'BIOS', 'map_name': 'bios7.Quinones'}
258 1
{'organism': 'BIOS', 'map_name': 'bios7.fa'}
372 8
{'organism': 'BIOS', 'map_name': 'bios7.AA'}
178 1
{'organism': 'BIOS', 'map_name': 'bios7.LipidIV'}
120 0
{'organism': 'BIOS', 'map_name': 'bios7.thf'}
73 0
{'organism': 'BIOS', 'map_name': 'bios7.Central'}
233 46
{'organism': 'BIOS', 'map_name': 'bios7.mycolate'}
193 2
{'organism': 'BIOS', 'map_name': 'bios7.Riboflavin'}
32 3
{'organism': 'BIOS', 'map_name': 'bios7.sugars'}
416 11
{'organism': 'BIOS', 'map_name': 'bios7.f430'}
141 0
{'organism': 'BIOS', 'map_name': 'bios7.Thiamine'}
156 2
{'organism': 'BIOS', 'map_name': 'bios7.phospholipids'}
69 0
{'organism': 'BIOS', 'map_name': 'bios7.Nucleotides'}
112 0
{'organism': 'BIOS', 'map_name': 'bios7.peptidoglycan'}
113 0


In [282]:
escher_map_data = read_json(ESCHER_HOME + '/maps/' + 'BIOS' + '/' + map_id + '.json')
escher_map = EscherMap(escher_map_data)
#escher_map.swap_ids(cpd_remap, rxn_remap)

In [286]:
cpd_ids = set(map(lambda o : o['bigg_id'], escher_map.metabolites))
rxn_ids = set(map(lambda o : o['bigg_id'], escher_map.reactions))
delete_rxn_ids = rxn_ids - set(rxn_remap)

In [328]:
def delete_orphan_metabolites(escher_map):
    deleted = []
    return deleted

def delete_nodes(escher_map, uids):
    deleted = []
    
    keep_nodes = dict(
        filter(
            lambda kv : kv[0] not in uids, 
            escher_map.escher_graph['nodes'].items())
    )
    
    escher_map.escher_graph['nodes'] = keep_nodes
    
    return deleted

def remove_reactions(escher_map, reactions, remove_orphans=False):
    deleted = []
    
    keep_reactions = {}
    non_orphan_nodes = set()
    
    ids = reactions
    
    marker_ids = set()
    
    marker_types = ['multimarker', 'midmarker']
    
    for uid in escher_map.escher_graph['reactions']:
        r = escher_map.escher_graph['reactions'][uid]
        if not r['bigg_id'] in ids:
            keep_reactions[uid] = r
            for s in r['segments'].values():
                non_orphan_nodes.add(s['from_node_id'])
                non_orphan_nodes.add(s['to_node_id'])
    
    delete_node_ids = set(escher_map.escher_graph['nodes']) - non_orphan_nodes
    for uid in delete_node_ids:
        if escher_map.escher_graph['nodes'][uid]['node_type'] in marker_types:
            marker_ids.add(uid)
    
    if remove_orphans:
        delete_nodes(escher_map, delete_node_ids)
    else:
        delete_nodes(escher_map, marker_ids)
        
    escher_map.escher_graph['reactions'] = keep_reactions
    
    return deleted

remove_reactions(escher_map, delete_rxn_ids, True)
#get_graph_reaction(escher_map, "DAPDC")

[]

In [303]:
def get_graph_reaction_by_uid(escher_map, uid):
    if uid in escher_map.escher_graph['reactions']:
        return escher_map.escher_graph['reactions'][uid]
    return None

def get_graph_reaction(escher_map, rxn_id):
    return list(
        filter(
            lambda o : o['bigg_id'] == rxn_id,
            escher_map.escher_graph['reactions'].values()
        )
    )

In [344]:
map_id = 'bios7.proteins'
escher_map_data = read_json(ESCHER_HOME + '/maps/' + 'BIOS' + '/' + map_id + '.json')
escher_map = EscherMap(escher_map_data)
cpd_ids = set(map(lambda o : o['bigg_id'], escher_map.metabolites))
rxn_ids = set(map(lambda o : o['bigg_id'], escher_map.reactions))
delete_rxn_ids = rxn_ids - set(rxn_remap)
remove_reactions(escher_map, delete_rxn_ids, True)
escher_map.swap_ids(cpd_remap, rxn_remap)
builder = build_escher_map(escher_map, model_json)
builder.display_in_notebook(enable_editing=False)

In [66]:
#builder.display_in_notebook(enable_editing=True)

In [48]:
mm = json.loads(model_json)

In [68]:
for r in mm['reactions']:
    if 'bio' in r['id']:
        print(r['id'])

bio1_biomass


In [73]:
rxn_var

{'biomass_ref': '~/fbamodel/biomasses/id/bio1',
 'class': 'unknown',
 'lowerBound': 0,
 'max': 1000,
 'min': 0,
 'other_max': [],
 'other_min': [],
 'other_values': [],
 'upperBound': 1000,
 'value': 0.697833,
 'variableType': 'biomassflux'}

In [72]:
fba.data.keys()



dict_keys(['FBABiomassVariables', 'FBACompoundBounds', 'FBACompoundVariables', 'FBAConstraints', 'FBADeletionResults', 'FBAMetaboliteProductionResults', 'FBAMinimalMediaResults', 'FBAMinimalReactionsResults', 'FBAPromResults', 'FBAReactionBounds', 'FBAReactionVariables', 'FBATintleResults', 'MFALog', 'PROMKappa', 'QuantitativeOptimizationSolutions', '__VERSION__', 'additionalCpd_refs', 'allReversible', 'biomassRemovals', 'biomassflux_objterms', 'calculateReactionKnockoutSensitivity', 'comboDeletions', 'compoundflux_objterms', 'decomposeReversibleDrainFlux', 'decomposeReversibleFlux', 'defaultMaxDrainFlux', 'defaultMaxFlux', 'defaultMinDrainFlux', 'drainfluxUseVariables', 'fbamodel_ref', 'findMinimalMedia', 'fluxMinimization', 'fluxUseVariables', 'fva', 'gapfillingSolutions', 'geneKO_refs', 'id', 'inputfiles', 'maximizeActiveReactions', 'maximizeObjective', 'media_list_refs', 'media_ref', 'minimizeErrorThermodynamicConstraints', 'minimize_reaction_costs', 'minimize_reactions', 'noErrorT

In [164]:
#modelseed_local.get_seed_compound('cpd15268').data['name']

In [171]:

#map_reaction

In [19]:
ls /Users/fliu/workspace/jupyter/data/escher/maps/BIOS

bios4.Porphyrin Metabolism.json  bios7.f430.json
bios7.AA.json                    bios7.fa.json
bios7.Central.json               bios7.mycolate.json
bios7.LipidIV.json               bios7.peptidoglycan.json
bios7.Nucleotides.json           bios7.proteins.json
bios7.Quinones.json              bios7.sugars.json
bios7.Riboflavin.json            bios7.thf.json
bios7.Thiamine.json              proteins.json


In [251]:
cobra_model.metabolites.cpd00346_c0.summary()

PRODUCING REACTIONS -- L-Aspartate4-semialdehyde (cpd0034...)
-------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
76%     3.72  rxn0130...  cpd00003_c0 + cpd00227_c0 --> cpd00004_c0 + cpd...
24%     1.2   rxn0164...  cpd00006_c0 + cpd00009_c0 + cpd00346_c0 <=> cpd...

CONSUMING REACTIONS -- L-Aspartate4-semialdehyde (cpd0034...)
-------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
100%    4.91  rxn0164...  cpd00020_c0 + cpd00346_c0 --> 2 cpd00001_c0 + c...


'../bin/escher/maps/bios7.Central.html'

In [ ]:
import os
import shutil
base = '/Users/fliu/workspace/jupyter/data/escher/maps/ModelSEED'
for f in os.listdir(base):
    src = base + '/' + f
    escher_json = read_json(src)
    if not 'authors' in escher_json[0]:
        escher_json[0]['authors'] = []
    if not 'Matthew Hoffman' in escher_json[0]['authors']:
        escher_json[0]['authors'].append('Matthew Hoffman')
    escher_json[0]['homepage'] = 'http://modelseed.org'
    escher_json[0]['map_description'] = f[10:-5]
    escher_json[0]['description'] = f[10:-5] #remove ModelSEED. and .json
    escher_json[0]['map_name'] = f[:-5] #remove .json
    write_json(escher_json, src)
    
    #dst = base + '/ModelSEED.' + f
    #shutil.move(src, dst)